# Models

| Model                  | Description                                                                                           | Context window | Max output tokens | Training data     | 1M Tokens(USD)  |
|------------------------|-------------------------------------------------------------------------------------------------------|----------------|-------------------|-------------------|----|
| gpt-4o            | GPT-4o: Our high-intelligence flagship model for complex, multi-step tasks. GPT-4o is cheaper and faster than GPT-4 Turbo. Currently points to gpt-4o-2024-05-13. | 128,000 tokens | 4,096 tokens     | Up to Oct 2023    | $5.00 input, $15.00 output |
| gpt-4o-mini            | Our affordable and intelligent small model for fast, lightweight tasks. GPT-4o mini is cheaper and more capable than GPT-3.5 Turbo. Currently points to gpt-4o-mini-2024-07-18. | 128,000 tokens | 16,384 tokens     | Up to Oct 2023    | $0.15 input, $0.60 output |

In [8]:
import json
import sympy as sp
import numpy as np
import scipy as sc
from openai import OpenAI
from textwrap import dedent
from dotenv import load_dotenv

load_dotenv()

client = OpenAI()

# MODEL = "gpt-4o" # expensive, larger model
MODEL = "gpt-4o-mini" # affordable, smaller model

In [11]:
math_tutor_prompt = '''
    You are a helpful math tutor. You will be provided with a math problem,
    and your goal will be to output a step by step solution, along with a final answer.
    For each step, just provide the output as an equation use the explanation field to detail the reasoning.
    If the problem is too hard, you can use python packages like sympy, numpy, scipy or even scikit-learn to help you solve it.
    You have a code interpreter available to you, so you can use it to help you solve the problem.
'''

user_question = "How can I solve 8x + 7 = -23"

In [10]:
from IPython.display import Math, display

def print_math_solution(response):
    result = json.loads(response)
    steps = result['steps']
    final_answer = result['final_answer']
    for i in range(len(steps)):
        print(f"Step {i+1}: {steps[i]['explanation']}\n")
        display(Math(steps[i]['output']))
        print("\n")
    print(f"Final Answer: {final_answer}")
    display(Math(final_answer))

# Solution using chat.completion

In [24]:
from pydantic import BaseModel
from typing import List

class MathReasoning(BaseModel):
    class Step(BaseModel):
        step_num: int
        explanation: str
        output: str

    steps: List[Step]
    final_answer: str

def get_math_solution(question: str) -> MathReasoning:
    completion = client.beta.chat.completions.parse(
        model=MODEL,
        messages=[
            {"role": "system", "content": dedent(math_tutor_prompt)},
            {"role": "user", "content": question}
        ],
        response_format=MathReasoning
    )
    return completion.choices[0].message

In [25]:
def print_reasoning(result: MathReasoning) -> None:
    for i, step in enumerate(sorted(result.steps, key=lambda x: x.step_num)):
        print(f"Step {i+1}: {step.explanation}\n")
        display(Math(step.output))
        print("\n")
    print(f"Final Answer: {result.final_answer}")

In [26]:
result = get_math_solution(user_question).parsed

In [ ]:
print_reasoning(result)

# Solution using assistant

In [19]:
import time

def check_run_status(thread_id, run_id):
    while True:
        # Retrieve the current status of the run
        run = client.beta.threads.runs.retrieve(
            thread_id=thread_id,
            run_id=run_id
        )
        
        # Check if the run requires an action
        if run.status == "requires_action":
            print("The assistant is waiting for an action.")
            return run
        
        elif run.status == "completed":
            print("The task is completed.")
            return run
        
        # If it's still in progress, wait and check again
        elif run.status in ["in_progress", "queued"]:
            print("Task is still in progress...")
            time.sleep(2)
    print(f"Run status: {run.status}")


In [33]:
import sys 
from code_interpreter import CodeInterpreter

coder = CodeInterpreter()

def call_coder(code: str) -> str:
    output = coder.execute(code)
    sys.stdout = sys.__stdout__
    return output

call_coder("x = 5")

'Finished'

In [18]:
assistant = client.beta.assistants.create(
    name = "Math Tutor",
    instructions = dedent(math_tutor_prompt),
    tools = [
        {
            "type": "function",
            "function": {
                "name": "call_coder",
                "description": "Call a python code interpreter to run code",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "code": {
                            "type": "string",
                            "description": "The python code to run"
                        }
                    },
                    "required": ["code"]
                }
            }
        }
    ],
    model = MODEL
)

thread = client.beta.threads.create()

message = client.beta.threads.messages.create(
    thread_id = thread.id,
    role = "user",
    content = user_question,    
)

In [20]:
run = client.beta.threads.runs.create_and_poll(
    thread_id = thread.id,
    assistant_id = assistant.id,   
)

run = check_run_status(thread.id, run.id)

In [22]:
messages = client.beta.threads.messages.list(thread_id=thread.id)

for message in messages:
    print(f"{message.role}: {message.content}")

In [41]:
def display_messages(messages):
    for message in messages:
        print(f"Role: {message.role}\nContent: {message.content}\n{'-'*40}")

# Usage
messages = client.beta.threads.messages.list(thread_id=thread.id)
display_messages(messages)

In [42]:
# | Model                  | Description                                                                                           | Context window | Max output tokens | Training data     | 1M Tokens(USD)  |
# |------------------------|-------------------------------------------------------------------------------------------------------|----------------|-------------------|-------------------|----|
# | gpt-4o            | GPT-4o: Our high-intelligence flagship model for complex, multi-step tasks. GPT-4o is cheaper and faster than GPT-4 Turbo. Currently points to gpt-4o-2024-05-13. | 128,000 tokens | 4,096 tokens     | Up to Oct 2023    | $5.00 input, $15.00 output |
# | gpt-4o-mini            | Our affordable and intelligent small model for fast, lightweight tasks. GPT-4o mini is cheaper and more capable than GPT-3.5 Turbo. Currently points to gpt-4o-mini-2024-07-18. | 128,000 tokens | 16,384 tokens     | Up to Oct 2023    | $0.15 input, $0.60 output |

import sys
import json
import time
from openai import OpenAI
from textwrap import dedent
from dotenv import load_dotenv
from sympy import sympify, pretty
from code_interpreter import CodeInterpreter

load_dotenv()

client = OpenAI()
coder = CodeInterpreter()

# MODEL = "gpt-4o" # expensive, larger model
MODEL = "gpt-4o-mini"  # affordable, smaller model


def print_math_solution(response):
    result = json.loads(response)
    steps = result["steps"]
    final_answer = result["final_answer"]
    for i in range(len(steps)):
        print(f"Step {i+1}: {steps[i]['explanation']}\n")
        print(pretty(sympify(steps[i]["output"])))
        print("\n")
    print(f"Final Answer: {final_answer}")
    print(pretty(sympify(final_answer)))


def call_coder(code: str) -> str:
    output = coder.execute(code)
    sys.stdout = sys.__stdout__
    return output


tools = [
    {
        "type": "function",
        "function": {
            "name": "call_coder",
            "description": "Call a python code interpreter to run code.",
            "parameters": {
                "type": "object",
                "properties": {
                    "code": {
                        "type": "string",
                        "description": "The python code to run.",
                    }
                },
                "required": ["code"],
            },
        },
    }
]

# user_question = "How can I solve 8x + 7 = -23"
user_question = "Evaluate $\\int sin^5(x) dx$"

math_tutor_prompt = """
    You are a helpful math tutor. You will be provided with a math problem,
    and your goal will be to output a step by step solution, along with a final answer.
    For each step, just provide the output as an equation use the explanation field to detail the reasoning.
    If the problem is too hard, you can use python packages like sympy, numpy, scipy or even scikit-learn to help you solve it.
    You have a code interpreter available to you, so you can use it to help you solve the problem.
"""

messages = [
    {
        "role": "assistant",
        "content": math_tutor_prompt,
    },
    {
        "role": "user",
        "content": user_question,
    },
]

for message in messages:
    print(f"{message['role']}: {message['content']}")

response = client.chat.completions.create(
    model=MODEL,
    messages=messages,
    tools=tools,
)

AttributeError: 'Completions' object has no attribute 'create'